<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Parkings-de-Strasbourg" data-toc-modified-id="Parkings-de-Strasbourg-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Parkings de Strasbourg</a></span></li><li><span><a href="#Lecture-URL-Stations-et-sauvegarde" data-toc-modified-id="Lecture-URL-Stations-et-sauvegarde-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Lecture URL Stations et sauvegarde</a></span></li><li><span><a href="#Lecture-Stations" data-toc-modified-id="Lecture-Stations-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Lecture Stations</a></span></li><li><span><a href="#Livraison-des-paniers" data-toc-modified-id="Livraison-des-paniers-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Livraison des paniers</a></span></li></ul></div>

In [1]:
import os
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

In [3]:
spark

In [3]:
!dir

 Le volume dans le lecteur C s'appelle Windows
 Le num‚ro de s‚rie du volume est 56E1-170F

 R‚pertoire de C:\Formation\spark

13/12/2021  23:39    <DIR>          .
13/12/2021  23:39    <DIR>          ..
13/12/2021  22:15    <DIR>          .ipynb_checkpoints
07/12/2021  10:05            28ÿ148 01-API.DataFrames-colonnes-projections.ipynb
07/12/2021  10:29            17ÿ476 02-API.DataFrames-Restrictions.ipynb
07/12/2021  17:15            51ÿ239 03-API.DataFrames-agregations.ipynb
07/12/2021  11:28            32ÿ298 04-API.DataFrames-jointures.ipynb
07/12/2021  11:37            44ÿ116 05-API.DataFrames-donnees-manquantes.ipynb
07/12/2021  12:07            25ÿ872 05-API.DataFrames-SQL.ipynb
07/12/2021  11:54               674 derby.log
13/12/2021  20:50    <DIR>          donnees
13/12/2021  17:34               857 get-stations.py
13/12/2021  17:39            24ÿ597 L'ex‚cution d'une application pyspark .ipynb
13/12/2021  23:39            48ÿ036 LectureEcritureJSON.ipynb
07/12/2021  11:54

In [4]:
!mkdir donnees
!mkdir donnees\json

Un sous-r‚pertoire ou un fichier donnees existe d‚j….
Un sous-r‚pertoire ou un fichier donnees\json existe d‚j….


In [5]:
import json
import urllib.request
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Parkings de Strasbourg

In [6]:
url = 'https://data.strasbourg.eu/api/records/1.0/search/?dataset=occupation-parkings-temps-reel&q=&lang=fr&rows=28'

In [8]:
donnees = spark.read.json(sc.parallelize(
            [x['fields'] for x in
                  json.loads(urllib.request.urlopen(url
                         ).read().decode())['records']
                ]))
donnees.select('*').show(5)

+----+---------------+-----+-----------+----------+-----+--------------------+-----+
|etat|etat_descriptif|ident|    idsurfs|infousager|libre|         nom_parking|total|
+----+---------------+-----+-----------+----------+-----+--------------------+-----+
|   1|         Ouvert|   10| 1382_DEP_8|       510|  518|  Parking Gare Wodli|  955|
|   2|          Fermé|    1|1400_DEP_26|     FERME| 3172|      Parking Zénith| 3175|
|   2|          Fermé|    3| 1378_DEP_4|     FERME|    0|Parking Centre Op...|  460|
|   1|         Ouvert|    4| 1376_DEP_2|       450|  457|   Parking Baggersee|  460|
|   1|         Ouvert|    2| 1377_DEP_3|       150|  151|   Parking Bateliers|  267|
+----+---------------+-----+-----------+----------+-----+--------------------+-----+
only showing top 5 rows



In [9]:
fichier = 'donnees/json/parking_stras_json'
format  = 'json'
donnees.write.mode('overwrite').format(format).save(fichier)

In [10]:
parkings = spark.read.format('json').option('mergeSchema', 'true').option('inferSchema','true').load('donnees/json/parking_stras_json').cache()
parkings.printSchema()

root
 |-- etat: long (nullable = true)
 |-- etat_descriptif: string (nullable = true)
 |-- ident: long (nullable = true)
 |-- idsurfs: string (nullable = true)
 |-- infousager: string (nullable = true)
 |-- libre: long (nullable = true)
 |-- nom_parking: string (nullable = true)
 |-- total: long (nullable = true)



In [11]:
parkings.show(3)

+----+---------------+-----+-----------+----------+-----+--------------------+-----+
|etat|etat_descriptif|ident|    idsurfs|infousager|libre|         nom_parking|total|
+----+---------------+-----+-----------+----------+-----+--------------------+-----+
|   1|         Ouvert|   14|1385_DEP_11|       380|  385|Parking P2 Sebast...|  496|
|   1|         Ouvert|   15|1393_DEP_19|        80|   84|Parking Robertsau...|   92|
|   1|         Ouvert|   16|1389_DEP_15|     LIBRE|  190|Parking Krimmeri ...|  190|
+----+---------------+-----+-----------+----------+-----+--------------------+-----+
only showing top 3 rows



# Lecture URL Stations et sauvegarde

In [161]:
schema = StructType([
        StructField('number',                StringType()  , True),
        StructField('contract_name',         StringType()  , True),
        StructField('name',                  StringType()  , True),
        StructField('address',               StringType()  , True),
        StructField('position',              StringType()  , True),
        StructField('banking',               StringType()  , True),
        StructField('bonus',                 StringType()  , True),
        StructField('bike_stands',           IntegerType() , True),
        StructField('available_bike_stands', IntegerType() , True),
        StructField('available_bikes',       IntegerType() , True),
        StructField('status',                StringType()  , True),
        StructField('last_update',           StringType()  , True)])

In [162]:
API_KEY = "e89ad7b276374a9d0a5fd75f8a9402c0e765cc08"
url = "https://api.jcdecaux.com/vls/v1/stations?apiKey={}".format(API_KEY)
donnees =  spark.createDataFrame(sc.parallelize([
                x for x in json.loads(urllib.request.urlopen(url).read().decode())
                ]),schema).select('number',               
                                  'contract_name',        
                                  'name',                 
                                  'address',              
                                  'position',             
                                  'banking',              
                                  'bonus',                
                                  'bike_stands',          
                                  'available_bike_stands',
                                  'available_bikes',      
                                  'status',               
                                  from_unixtime(col('last_update')/1000).cast(TimestampType()).alias('last_update'))

donnees.toPandas().head()

,number,contract_name,name,address,position,banking,bonus,bike_stands,available_bike_stands,available_bikes,status,last_update
0,9087,marseille,9087-MAZARGUES,MAZARGUES - ROND POINT DE MAZARGUES (OBELISQUE),"{lng=5.403244616491982, lat=43.250903869637334}",true,false,21,18,3,OPEN,2021-12-13 22:53:14
1,55,toulouse,00055 - SAINT-SERNIN - G. ARNOULT,2 RUE GATIEN ARNOULT,"{lng=1.4410035987262, lat=43.6089519604964}",true,false,15,5,10,OPEN,2021-12-13 22:45:58
2,59,ljubljana,LIDL BEŽIGRAD,Bežigrad 11,"{lng=14.506854, lat=46.063797}",false,false,20,18,2,OPEN,2021-12-13 22:48:10
3,2010,lyon,2010 - CONFLUENCE / DARSE,ANGLE ALLEE ANDRE MURE ET QUAI ANTOINE RIBOUD,"{lng=4.815747, lat=45.743317}",true,false,22,3,17,OPEN,2021-12-13 22:45:40
4,5015,lyon,5015 - FULCHIRON,Devant le n°41 rue de la Quarantaine,"{lng=4.821662, lat=45.75197}",true,false,19,0,19,OPEN,2021-12-13 22:45:11


In [163]:
donnees.printSchema()

root
 |-- number: string (nullable = true)
 |-- contract_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- position: string (nullable = true)
 |-- banking: string (nullable = true)
 |-- bonus: string (nullable = true)
 |-- bike_stands: integer (nullable = true)
 |-- available_bike_stands: integer (nullable = true)
 |-- available_bikes: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- last_update: timestamp (nullable = true)



In [164]:
fichier = 'donnees/json/stations'
format  = 'json'
donnees.write.mode('overwrite').format(format).save(fichier)

# Lecture Stations

In [166]:
schema = StructType([
        StructField('number'               , StringType()   , True),
        StructField('contract_name'        , StringType()   , True),
        StructField('name'                 , StringType()   , True),
        StructField('address'              , StringType()   , True),
        StructField('position'             , StringType()   , True),
        StructField('banking'              , StringType()   , True),
        StructField('bonus'                , StringType()   , True),
        StructField('bike_stands'          , IntegerType()  , True),
        StructField('available_bike_stands', IntegerType()  , True),
        StructField('available_bikes'      , IntegerType()  , True),
        StructField('status'               , StringType()   , True),
        StructField('last_update'          , TimestampType(), True)])

In [167]:
stations = spark.read.format('json').option('mergeSchema', 'true').schema(schema).load('donnees/json/stations').cache()
stations.printSchema()

root
 |-- number: string (nullable = true)
 |-- contract_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- position: string (nullable = true)
 |-- banking: string (nullable = true)
 |-- bonus: string (nullable = true)
 |-- bike_stands: integer (nullable = true)
 |-- available_bike_stands: integer (nullable = true)
 |-- available_bikes: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- last_update: timestamp (nullable = true)



In [169]:
stations.toPandas().head()

,number,contract_name,name,address,position,banking,bonus,bike_stands,available_bike_stands,available_bikes,status,last_update
0,5008,lyon,5008 - SAINTE-IRÉNÉE,Place Sainte Irénée,"{lng=4.812427, lat=45.754978}",false,true,20,15,5,OPEN,2021-12-13 22:48:25
1,16,namur,016 - PLACE DE WALLONIE,016 - PLACE DE WALLONIE - PLACE DE LA WALLONIE,"{lng=4.87266, lat=50.45602}",true,false,15,7,8,OPEN,2021-12-13 22:52:26
2,14,nantes,014-SAINT FÉLIX,"35, Boulevard Amiral Courbet","{lng=-1.556133, lat=47.230334}",false,false,14,10,4,OPEN,2021-12-13 22:54:55
3,141,bruxelles,141 - GARE DU NORD/NOORDSTATION,GARE DU NORD/NOORDSTATION - PLACE SIMON BOLIVA...,"{lng=4.358116, lat=50.860652}",false,false,42,12,30,OPEN,2021-12-13 22:54:06
4,248,bruxelles,248 - PRINCE DE LIGNE / PRINCE LIJN,PRINCE DE LIGNE / PRINCE LIJN - CHEE DE WATERL...,"{lng=4.373631, lat=50.795704}",false,false,22,6,16,OPEN,2021-12-13 22:51:07


# Livraison des paniers

In [130]:
url = 'https://data.strasbourg.eu/api/records/1.0/search/?dataset='
dataset = 'livraison_de_paniers'
urlDS = url+dataset
urlDS = urlDS+"&q=&lang=fr&rows="+str(json.loads(urllib.request.urlopen(urlDS).read().decode())['nhits'])
donnees = spark.read.json(sc.parallelize(
            [x['fields'] for x in
                  json.loads(urllib.request.urlopen(urlDS
                         ).read().decode())['records']
                ]))

fichier = 'donnees/json/'+dataset
format  = 'json'
donnees.write.mode('overwrite').format(format).save(fichier)

In [131]:
donnees.toPandas().head()

,adresse_de_distribution,agriculture_biologique,commune_de_distribution,date_maj,exploitation,formules,geo_point_2d,jours_et_horaires,lieu,produits,quartier,sig_id,site_web,telephone
0,Place de la République,AB,Bischheim,02/09/2019,Ferme Herrmann,Sans abonnement,"[48.613144, 7.757405]",Mardi de 17h à 18h30,Parking Place de la République,Fruits et légumes,None,2,www.fruits-herrmann.fr/,03 88 70 57 10
1,12 rue Antoine Freyermuth,None,Lingolsheim,02/09/2019,La Ruche qui dit Oui,Sans abonnement,"[48.568137, 7.697459]",Jeudi de 18h à 19h,None,Multi-produits,None,12,www.laruchequiditoui.fr/fr,None
2,11 rue Boistel,None,Plobsheim,02/09/2019,La Ruche qui dit Oui,Sans abonnement,"[48.471986, 7.726139]",Vendredi de 18h30 à 19h30,None,Multi-produits,None,17,www.laruchequiditoui.fr/fr,None
3,Rue des Pompiers,None,Schiltigheim,02/09/2019,Ferme Hager,Sans abonnement,"[48.606655, 7.749879]",Mardi de 16h à 18h30,Maison du Jeune Citoyen,"Fruits et légumes, oeufs",None,19,www.fermehager.fr,03 88 51 28 37
4,1 Allée d'Athènes,AB,Schiltigheim,02/09/2019,Les Jardins de la Montagne Verte,3 formules d'abonnement,"[48.607784, 7.708715]",Mardi de 12h à 17h,IUT Louis Pasteur,Fruits et légumes,None,23,www.jardins-montagne-verte.com/,03 88 29 36 33


In [12]:
paniers = spark.read.format('json').option('mergeSchema', 'true').option('inferSchema','true').load('donnees/json/livraison_de_paniers').cache()
paniers.printSchema()

root
 |-- adresse_de_distribution: string (nullable = true)
 |-- agriculture_biologique: string (nullable = true)
 |-- commune_de_distribution: string (nullable = true)
 |-- date_maj: string (nullable = true)
 |-- exploitation: string (nullable = true)
 |-- formules: string (nullable = true)
 |-- geo_point_2d: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- jours_et_horaires: string (nullable = true)
 |-- lieu: string (nullable = true)
 |-- produits: string (nullable = true)
 |-- quartier: string (nullable = true)
 |-- sig_id: string (nullable = true)
 |-- site_web: string (nullable = true)
 |-- telephone: string (nullable = true)



In [14]:
paniers.toPandas().head()

,adresse_de_distribution,agriculture_biologique,commune_de_distribution,date_maj,exploitation,formules,geo_point_2d,jours_et_horaires,lieu,produits,quartier,sig_id,site_web,telephone
0,Chemin du Grossroethig,AB,Strasbourg,02/09/2019,Les Jardins de la Montagne Verte,3 formules d'abonnement,"[48.5719394111, 7.70795030284]",Mardi au vendredi de 11h à 17h,Jardins de la Montagne Verte,Fruits et légumes,Koeningshoffen,66,www.jardins-montagne-verte.com/,03 88 29 36 33
1,3 rue Mgr Hoch,AB,Strasbourg,02/09/2019,Ried Oasis,3 formules d'abonnement,"[48.576182, 7.699618]",Jeudi de 16h30 à 18h,Ecole Joie de Vivre,"Fruits, légumes, pain et oeufs",Koeningshoffen,70,www.riedoasis.fr/,03 88 98 49 32
2,68 rue de Zurich (devant Krutt'herbes),AB,Strasbourg,02/09/2019,Ferme de Truttenhausen,2 formules d'abonnement,"[48.578571, 7.75894]",Le mardi de 18h00 à 19h30,AMAP Truttenhausen,"Fruits et légumes, pain et produits laitiers",Krutenau,71,www.truttenhausen.jimdo.com/vente-de-produits/...,03 88 08 24 33
3,"Centre socio-culturel CARDEK, 1 place des Orph...",AB,Strasbourg,02/09/2019,Le Panier du Jardin d'Agnès,3 abonnements,"[48.578822, 7.753991]",Mercredi de 18h15 à 19h30,La Carotte sociale et solidaire,"Fruits et légumes, œufs",Krutenau,72,www.lejardindagnes.fr/,06 08 21 54 68
4,13 rue d'Austerlitz,AB,Strasbourg,02/09/2019,Vegetal Respekt,Sans abonnement,"[48.578576, 7.752399]",Vendredi 15h à 19h30 et samedi 9h à 20h,HAPPY FLEURS,"Fruits, légumes et aromates",Krutenau,73,https://ma-ferme-bio.com/,03 90 40 66 35
